In [3]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
import tensorflow_probability as tfp

import os
import io
import base64
import time
import glob
from IPython.display import HTML

import pybullet_envs

%matplotlib inline

In [4]:
def make_env(env_name, seed=1888):
    # remove time limit wrapper from environment
    env = gym.make(env_name).unwrapped
    if seed is not None:
        env.seed(seed)
    return env

In [6]:

env_name = 'HalfCheetahBulletEnv-v0' #  'HopperBulletEnv-v0' # 'AntBulletEnv-v0' 
seed = 1888
env = make_env(env_name,seed )

print(f'env : {env}')
state_shape, action_shape = env.observation_space.shape, env.action_space.shape
print('State shape: {}'.format(state_shape))
print('Action shape: {}'.format(action_shape))
print(f'action space {env.action_space} observation space : {env.observation_space}')
print(f'action space bound :{env.action_space.low}, {env.action_space.high}')
act_limit = env.action_space.high[0]
act_dim = env.action_space.shape[0]
print(f'action limit = {act_limit} dimension {act_dim}')
state_dim = env.observation_space.shape[0]
print(state_dim)

env : <HalfCheetahBulletEnv<HalfCheetahBulletEnv-v0>>
State shape: (26,)
Action shape: (6,)
action space Box(-1.0, 1.0, (6,), float32) observation space : Box(-inf, inf, (26,), float32)
action space bound :[-1. -1. -1. -1. -1. -1.], [1. 1. 1. 1. 1. 1.]
action limit = 1.0 dimension 6
26


In [7]:
class Policy_p(tf.keras.Model):
    def __init__(self, act_dim, act_limit):
        super(Policy_p, self).__init__()
        self.act_limit = act_limit
        self.fc1 = tf.keras.layers.Dense(400, activation="relu")
        self.fc2 = tf.keras.layers.Dense(300, activation="relu")
        self.mean = tf.keras.layers.Dense(act_dim, activation = 'tanh')
        self.log_std_dev = tf.keras.layers.Dense(act_dim, activation='linear')
    
    def call(self, s):
        x = self.fc1(s)
        x = self.fc2(x)

        mu = self.mean(x)
        log_std = self.log_std_dev(x)
        return mu, log_std
    
    def act(self, state, evaluate=False):

        mu, log_std = self.call(state)
        std_dev = tf.exp(log_std)
        
        normal = tfp.distributions.Normal(mu,std_dev)
        z = normal.sample() 
        actions = tf.clip_by_value(z,-self.act_limit,self.act_limit)
        probs = normal.prob(actions)
        
        return actions, probs, mu, log_std

In [8]:
class Critic(tf.keras.Model):
    def __init__(self):
        super(Critic, self).__init__()
        self.fc1 = tf.keras.layers.Dense(512, activation="relu")
        self.fc2 = tf.keras.layers.Dense(512, activation="relu")
        self.Q = tf.keras.layers.Dense(1)
    
    def call(self, s, a):
        x = tf.concat([s,a], axis=-1)
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.Q(x)
        return tf.squeeze(q, -1)

In [9]:
class ReplayBuffer:
    def __init__(self, size=1e6):
        self.size = size #max number of items in buffer
        self.buffer =[] #array to holde buffer
        self.next_id = 0
    
    def __len__(self):
        return len(self.buffer)
    
    def add(self, state, action, mu, log_std,  reward, next_state, done):
        item = (state, action, mu, log_std, reward, next_state, done)
        if len(self.buffer) < self.size:
            self.buffer.append(item)
        else:
            self.buffer[self.next_id] = item
        self.next_id = (self.next_id + 1) % self.size
        
    def sample(self, batch_size=32):
        idxs = np.random.choice(len(self.buffer), batch_size)
        samples = [self.buffer[i] for i in idxs]
        states, actions,  mus, log_stds, rewards, next_states, done_flags = list(zip(*samples))
        return np.array(states,np.float32), np.array(actions,np.float32), np.array(mus, np.float32), np.array(log_stds, np.float32),  np.array(rewards,np.float32), np.array(next_states,np.float32), np.array(done_flags)
    
    def to_tensors(self, state_dim, act_dim):
        states, actions, mus,log_stds, rewards, next_states, done_flags = self.sample(32)
        states = np.array(states,np.float32)
        states = np.reshape(states, (-1, state_dim))
    
        actions = np.reshape(actions, (-1,act_dim))
        mus = np.reshape(mus, (-1,act_dim))
        log_stds = np.reshape(log_stds, (-1,act_dim))

        rewards = np.reshape(rewards,(-1,1))
        rewards = rewards.squeeze()

        next_states = np.array(next_states,np.float32)
        next_states = np.reshape(next_states, (-1, state_dim))
    
        done_flags = np.reshape(done_flags, (-1,1))
        done_flags = np.squeeze(done_flags)

        ##print(f' states {states} actions : {actions} rewards : {rewards}:  next_states {next_states} dones flags : {done_flags}')

        state_ts = tf.convert_to_tensor(states, dtype= tf.float32)
        action_ts = tf.convert_to_tensor(actions, dtype=tf.float32)
        mus_ts = tf.convert_to_tensor(mus, dtype = tf.float32)
        log_stds_ts = tf.convert_to_tensor(log_stds, dtype = tf.float32)

        reward_ts = tf.convert_to_tensor(rewards, dtype=tf.float32)
        next_state_ts = tf.convert_to_tensor(next_states,dtype=tf.float32)
    
        return state_ts, action_ts, mus_ts, log_stds_ts, reward_ts, next_state_ts, done_flags
    
    def initialize_replay_buffer(self,env, n_steps = 100):
        state = env.reset()
        for _ in range(n_steps):
            action = env.action_space.sample()
            next_state, reward, done, _ = env.step(action)
            buffer.add(state, action,action,action, reward, next_state, done) #action values added as place holders during initialization
            if done:
                state = env.reset()
            state = next_state
buffer = ReplayBuffer(100000)

In [10]:
#TD3 with KL regularizer
class AgentTD3E:
    def __init__(self,env, act_dim, act_limit, state_dim, learning_rate = 1e-3, gamma = 0.99, polyak = 0.95):
        self.dist = tfp.distributions.Normal(0,0.3)
        self.learning_rate_critic = 0.002
        self.learning_rate_policy = 1e-3
        self.polyak = polyak
        self.gamma = gamma
        self.act_dim = act_dim
        self.state_dim = state_dim
        self.act_limit = act_limit

        self.critic1 = Critic()
        self.critic2 = Critic()

        self.target_critic1 = Critic()
        self.target_critic2 = Critic()

        self.policy = Policy_p(act_dim,act_limit)
        self.target_policy = Policy_p(act_dim,act_limit)

        s = env.reset()
        a = env.action_space.sample()
        s = s[np.newaxis]

        _ = self.critic1(s,a[np.newaxis])
        _ = self.critic2(s,a[np.newaxis])

        _ = self.target_critic1(s,a[np.newaxis])
        _ = self.target_critic2(s,a[np.newaxis])

        _,_ = self.policy(s)
        _,_ = self.target_policy(s)

        self.target_critic1.set_weights(self.critic1.get_weights())
        self.target_critic2.set_weights(self.critic2.get_weights())
        self.target_policy.set_weights(self.policy.get_weights())

        self.target_critic1.trainable = False
        self.target_critic2.trainable = False
        self.target_policy.trainable = False

        self.policy_optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate_policy)
        self.critic_optimizer1 = tf.keras.optimizers.Adam(learning_rate=self.learning_rate_critic)
        self.critic_optimizer2 = tf.keras.optimizers.Adam(learning_rate=self.learning_rate_critic)
   
    def polyak_update(self, target_network, network):
        updated_model_weights = []
        for weights, target_weights in zip(network.get_weights(), target_network.get_weights()):
            new_weights = self.polyak * target_weights + ((1-self.polyak) * weights)
            updated_model_weights.append(new_weights)
        target_network.set_weights(updated_model_weights)
        
    @tf.function
    def compute_q_loss(self,states,actions, rewards, next_states, dones, gamma=0.99, eps=1e-6):
        with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            target_actions, _, _, _ = self.target_policy.act(next_states)
          
            target_actions = (tf.clip_by_value(target_actions, -self.act_limit, self.act_limit))
            target_qval1 = self.target_critic1(next_states,target_actions)
            target_qval2 = self.target_critic2(next_states,target_actions)

            qval1 = self.critic1(states, actions, training=True)
            qval2 = self.critic2(states, actions, training=True)

            target_next_qval = tf.math.minimum(target_qval1, target_qval2)
            target_qval = rewards + gamma * (1-dones) * target_next_qval
            
            critic_loss1 = tf.reduce_mean((target_qval - qval1)**2)
            critic_loss2 = tf.reduce_mean((target_qval - qval2)**2)

        grads1 = tape1.gradient(critic_loss1, self.critic1.trainable_variables)
        grads2 = tape2.gradient(critic_loss2, self.critic2.trainable_variables)
        
        self.critic_optimizer1.apply_gradients(zip(grads1,self.critic1.trainable_variables))       
        self.critic_optimizer2.apply_gradients(zip(grads2,self.critic2.trainable_variables))

        return critic_loss1, critic_loss2
    
    #@tf.function
    def compute_p_loss(self,states, mean_prev, log_std_prev, eps=1e-6):
        with tf.GradientTape() as tape:
            actions, _, mu_s, log_stds = self.policy.act(states) 


            kl = (log_stds - log_std_prev) + ( tf.square(tf.exp(log_std_prev)) + ((mu_s - mean_prev) **2) ) / (2 * tf.square(tf.exp(log_stds))) - 0.5 
            kl = tf.reduce_mean(tf.reduce_sum(kl, axis = -1))        
            policy_loss = - (self.critic1(states,actions) - (0.01 * kl))
            p_loss =  tf.math.reduce_mean(policy_loss)

        grads = tape.gradient(p_loss,self.policy.trainable_variables)
        self.policy_optimizer.apply_gradients(zip(grads,self.policy.trainable_variables))

        #print(f'states : {states} actions : {actions} policy_loss : {policy_loss} p_loss : {p_loss}') 

        return p_loss, kl

    def train_step(self,step):
        p_loss = 0
        states, actions, mus, log_stds, rewards, next_states, dones = buffer.to_tensors(self.state_dim,self.act_dim)
        #print(f'states: {states} actions : {actions} rewards : {rewards}')
        done_flags = np.array(dones,np.float32)
        c_loss1, c_loss2 = self.compute_q_loss(states,actions, rewards, next_states, done_flags)

        p_loss, kl = self.compute_p_loss(states, mus, log_stds)
                
        self.polyak_update(self.target_critic1, self.critic1)
        self.polyak_update(self.target_critic2, self.critic2)
        self.polyak_update(self.target_policy, self.policy)

        return p_loss, c_loss1, c_loss2, kl  

In [12]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      1.4.0
aiohttp                      3.8.4
aiosignal                    1.3.1
ale-py                       0.7.5
appdirs                      1.4.4
argcomplete                  3.0.8
asttokens                    2.2.1
astunparse                   1.6.3
async-timeout                4.0.2
atari-py                     1.2.2
atomicwrites                 1.4.1
attrs                        23.1.0
AutoROM                      0.4.2
AutoROM.accept-rom-license   0.6.1
backcall                     0.2.0
boto                         2.49.0
Box2D                        2.3.2
box2d-py                     2.3.5
cachetools                   5.3.0
certifi                      2022.12.7
cffi                         1.15.1
charset-normalizer           3.1.0
click                        8.1.3
cloudpickle                  2.2.1
colorama                     0.4.6
comm                         0.1.3
contour